# How to compute chemical properties

You have now learned how to create a {{ChemicalSystem}} object and {{ChemicalState}} objects representing the chemical state of that system. We will now demonstrate how thermodynamic and chemical properties of the system, its phases, and constituting species can be calculated using class {{ChemicalProps}}.

Let's start with a simple chemical system containing a single aqueous phase:

In [15]:
from reaktoro import *

db = PhreeqcDatabase("phreeqc.dat")

solution = AqueousPhase(speciate("H O Na Cl C"))

system = ChemicalSystem(db, solution)

Let's create a chemical state for this system representing a 1 molal NaCl solution with 0.7 molal dissolved CO{{_2}}:

In [16]:
state = ChemicalState(system)
state.temperature(60.0, "celsius")
state.pressure(15.0, "atm")
state.set("H2O", 1.0, "kg")
state.set("Na+", 1.0, "mol")
state.set("Cl-", 1.0, "mol")
state.set("CO2", 0.7, "mol")

equilibrate(state)

print(state)

+-----------------+--------------+------+
| Property        |        Value | Unit |
+-----------------+--------------+------+
| Temperature     |       333.15 |    K |
| Pressure        |  1.51988e+06 |   Pa |
| Charge:         | -1.03709e-16 |  mol |
| Element Amount: |              |      |
| :: H            |      111.012 |  mol |
| :: C            |          0.7 |  mol |
| :: O            |      56.9062 |  mol |
| :: Na           |            1 |  mol |
| :: Cl           |            1 |  mol |
| Species Amount: |              |      |
| :: CO3-2        |  4.94309e-11 |  mol |
| :: H+           |  0.000715616 |  mol |
| :: H2O          |      55.5055 |  mol |
| :: CO2          |     0.699284 |  mol |
| :: (CO2)2       |        1e-16 |  mol |
| :: HCO3-        |  0.000484623 |  mol |
| :: CH4          |        1e-16 |  mol |
| :: Cl-          |            1 |  mol |
| :: H2           |        1e-16 |  mol |
| :: Na+          |     0.999769 |  mol |
| :: NaCO3-       |  4.45274e-09 |

```{note}
We used the convenience method `equilibrate` above to quickly bring the {{ChemicalState}} object `state` into a state of chemical equilibrium. We will later learn how to use Reaktoro's Gibbs energy minimization algorithm to perform a wide variety of chemical equilibrium calculations using class {{EquilibriumSolver}} together with class {{EquilibriumSpecs}}, which allows us to enforce different equilibrium constraints at equilibrium (e.g., fixed pH, pE, Eh, gas fugacity, enthalpy, internal energy, volume, etc.).
```

We have seen that class {{ChemicalState}} is not responsible for computing thermodynamic and chemical properties of the system. It's primary use is for preparing initial states for chemical equilibrium/kinetics calculations and also to store the result of these computations.

Computing chemical properties of {{ChemicalSystem}} objects is a task for class {{ChemicalProps}}:

In [17]:
props = ChemicalProps(state)
print(props)

+----------------------------------------+--------------+-----------+
| Property                               |        Value |      Unit |
+----------------------------------------+--------------+-----------+
| Temperature                            |       333.15 |         K |
| Pressure                               |  1.51988e+06 |        Pa |
| Volume                                 |   0.00105949 |        m3 |
| Gibbs Energy                           |     -73283.2 |         J |
| Enthalpy                               |     -1608.04 |         J |
| Entropy                                |      215.144 |       J/K |
| Internal Energy                        |     -3218.33 |         J |
| Helmholtz Energy                       |     -74893.5 |         J |
| Charge                                 | -1.03709e-16 |       mol |
| Element Amount:                        |              |           |
| :: H                                   |      111.012 |       mol |
| :: C              

By default, when defining a chemical system using {{AqueousPhase}}, {{GaseousPhase}}, {{LiquidPhase}}, {{SolidPhase}}, {{MineralPhase}}, {{CondensedPhase}}, {{IonExchangePhase}}, etc., these objects are created with **ideal activity models**. That's why you should see unit activity coefficients above for all species. 

We will learn later how to specify more sophisticated and accurate activity models for each phase, but out of curiosity, we will define Pitzer's activity model from {cite:t}`Harvie1980,Harvie1984`. Let's also consider a gaseous phase and some mineral phases. For the gaseous phase, we shall use Peng-Robinson's equation of state {cite}`Peng1976` as its activity model. 

We will need to reconstruct the {{ChemicalSystem}} object, as well as the {{ChemicalState}} and {{ChemicalProps}} objects:

In [18]:
solution = AqueousPhase(speciate("H O Na Cl C Ca Si"))
solution.setActivityModel(ActivityModelPitzerHMW())

gases = GaseousPhase()
gases.setActivityModel(ActivityModelPengRobinson())

minerals = MineralPhases()

system = ChemicalSystem(db, solution, gases, minerals)

state = ChemicalState(system)
state.temperature(60.0, "celsius")
state.pressure(15.0, "atm")
state.set("H2O"    , 1.0, "kg")
state.set("Na+"    , 1.0, "mol")
state.set("Cl-"    , 1.0, "mol")
state.set("CO2"    , 0.7, "mol")
state.set("Calcite", 1.0, "g")
state.set("Quartz" , 1.0, "g")

equilibrate(state)

props = ChemicalProps(state)

print(props)

+----------------------------------------+--------------+-----------+
| Property                               |        Value |      Unit |
+----------------------------------------+--------------+-----------+
| Temperature                            |       333.15 |         K |
| Pressure                               |  1.51988e+06 |        Pa |
| Volume                                 |   0.00190225 |        m3 |
| Gibbs Energy                           |     -79641.2 |         J |
| Enthalpy                               |      5010.76 |         J |
| Entropy                                |      254.096 |       J/K |
| Internal Energy                        |      2119.58 |         J |
| Helmholtz Energy                       |     -82532.3 |         J |
| Charge                                 | -1.11419e-16 |       mol |
| Element Amount:                        |              |           |
| :: H                                   |      111.012 |       mol |
| :: C              

The results displayed above should now be a little more interesting.

Now consider a scenario where your {{ChemicalState}} object `state` has changed, for example 1 mol of NaCl has been added to it and the temperature has increased by 10 °C. The chemical properties of the system must then be updated as shown below:

In [19]:
state.temperature(70.0, "celsius")
state.add("Na+", 1.0, "mol")
state.add("Cl-", 1.0, "mol")

props.update(state)

print(props)

+----------------------------------------+--------------+-----------+
| Property                               |        Value |      Unit |
+----------------------------------------+--------------+-----------+
| Temperature                            |       343.15 |         K |
| Pressure                               |  1.51988e+06 |        Pa |
| Volume                                 |   0.00195689 |        m3 |
| Gibbs Energy                           |       -82231 |         J |
| Enthalpy                               |      8403.56 |         J |
| Entropy                                |      264.125 |       J/K |
| Internal Energy                        |      5429.33 |         J |
| Helmholtz Energy                       |     -85205.2 |         J |
| Charge                                 | -3.33463e-16 |       mol |
| Element Amount:                        |              |           |
| :: H                                   |      111.012 |       mol |
| :: C              

The printed table above shows many different properties. Let's now use some methods from class {{ChemicalProps}} to retrieve some specific properties for the system, its phases, and its species:

In [42]:
print("SYSTEM'S TEMPERATURE     :", props.temperature()                         , "K"     )
print("SYSTEM'S PRESSURE        :", props.pressure()                            , "Pa"    )
print("SYSTEM'S VOLUME          :", props.volume()                              , "m3"    )
print("AQUEOUS PHASE'S DENSITY  :", props.phaseProps("AqueousPhase").density()  , "kg/m3" )
print("AQUEOUS PHASE'S MASS     :", props.phaseProps("AqueousPhase").mass()     , "kg"    )
print("AQUEOUS PHASE'S VOLUME   :", props.phaseProps("AqueousPhase").volume()   , "m3"    )
print("AQUEOUS PHASE'S ENTHALPY :", props.phaseProps("AqueousPhase").enthalpy() , "J/mol" )
print("GASEOUS PHASE'S DENSITY  :", props.phaseProps("GaseousPhase").density()  , "kg/m3" )
print("GASEOUS PHASE'S MASS     :", props.phaseProps("GaseousPhase").mass()     , "kg"    )
print("GASEOUS PHASE'S VOLUME   :", props.phaseProps("GaseousPhase").volume()   , "m3"    )
print("GASEOUS PHASE'S ENTHALPY :", props.phaseProps("GaseousPhase").enthalpy() , "J/mol" )
print("ACTIVITY H+              :", props.speciesActivity("H+")                           )
print("ACTIVITY H+ (LOG10)      :", props.speciesActivityLg("H+")                         )
print("ACTIVITY COEFF. CO2      :", props.speciesActivityCoefficient("CO2")               )
print("ACTIVITY COEFF. HCO3-    :", props.speciesActivityCoefficient("HCO3-")             )
print("FUGACITY COEFF. CO2(g)   :", props.speciesActivityCoefficient("CO2(g)")            )
print("FUGACITY COEFF. H2O(g)   :", props.speciesActivityCoefficient("H2O(g)")            )
print("FUGACITY COEFF. O2(g)    :", props.speciesActivityCoefficient("O2(g)")             )

SYSTEM'S TEMPERATURE     : 343.15 K
SYSTEM'S PRESSURE        : 1.51988e+06 Pa
SYSTEM'S VOLUME          : 0.00195689 m3
AQUEOUS PHASE'S DENSITY  : 1058.56 kg/m3
AQUEOUS PHASE'S MASS     : 1.12676 kg
AQUEOUS PHASE'S VOLUME   : 0.00106443 m3
AQUEOUS PHASE'S ENTHALPY : 642.393 J/mol
GASEOUS PHASE'S DENSITY  : 24.6099 kg/m3
GASEOUS PHASE'S MASS     : 0.0219544 kg
GASEOUS PHASE'S VOLUME   : 0.000892096 m3
GASEOUS PHASE'S ENTHALPY : 8170.97 J/mol
ACTIVITY H+              : 1.51131e-05
ACTIVITY H+ (LOG10)      : -4.82065
ACTIVITY COEFF. CO2      : 1.37659
ACTIVITY COEFF. HCO3-    : 0.557782
FUGACITY COEFF. CO2(g)   : 0.947625
FUGACITY COEFF. H2O(g)   : 0.866819
FUGACITY COEFF. O2(g)    : 1.0112


```{tip}
Access the link {{ChemicalProps}} to find out all methods available in class `ChemicalProps` to get the exact data you need!
```